In [1]:
import prepare_functionscl as prep
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import sklearn.metrics as mets
from pydataset import data
import os
import acquire_copy as acq
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, \
confusion_matrix,\
classification_report

### Random Forest Exercises

In [2]:
df = prep.titanic

In [3]:
df.head()

,passenger_id,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,deck,embark_town,alone
0,0,0,3,male,22.0,1,0,7.2500,S,Third,NaN,Southampton,0
1,1,1,1,female,38.0,1,0,71.2833,C,First,C,Cherbourg,0
2,2,1,3,female,26.0,0,0,7.9250,S,Third,NaN,Southampton,1
3,3,1,1,female,35.0,1,0,53.1000,S,First,C,Southampton,0
4,4,0,3,male,35.0,0,0,8.0500,S,Third,NaN,Southampton,1


In [4]:
rose = prep.prep_titanic(df)

In [5]:
rose.head(2)

,survived,pclass,age,sibsp,parch,fare,alone,sex_female,sex_male,embark_town_Cherbourg,embark_town_Queenstown,embark_town_Southampton
0,0,3,22.0,1,0,7.2500,0,0,1,0,0,1
1,1,1,38.0,1,0,71.2833,0,1,0,1,0,0


In [10]:
train, val, test = prep.split_titanic(rose)

In [11]:
X_train, X_val, X_test = train.drop(columns='survived'), val.drop(columns='survived'), test.drop(columns='survived')
y_train, y_val, y_test = train['survived'], val['survived'], test['survived']

In [12]:
X_train.shape, X_val.shape, X_test.shape

((498, 11), (214, 11), (179, 11))

In [17]:
train.isna().sum()

survived                   0
pclass                     0
age                        0
sibsp                      0
parch                      0
fare                       0
alone                      0
sex_female                 0
sex_male                   0
embark_town_Cherbourg      0
embark_town_Queenstown     0
embark_town_Southampton    0
dtype: int64

#### Exercise 1
Fit the Random Forest classifier to your training sample and transform (i.e. make predictions on the training sample) setting the random_state accordingly and setting min_samples_leaf = 1 and max_depth = 10.

In [19]:
sherwood = RandomForestClassifier(min_samples_leaf = 1,
                                  max_depth = 10, random_state=123)

In [20]:
# Fit to train sets
sherwood.fit(X_train, y_train)

RandomForestClassifier(max_depth=10, random_state=123)

In [21]:
y_preds= sherwood.predict(X_train)

#### Exercise 2


Evaluate your results using the model score, confusion matrix, and classification report.


In [23]:
# Model Accuracy score
print(f' Model Score: {sherwood.score(X_train, y_train):.2%}')

 Model Score: 96.79%


In [24]:
classrep = classification_report(y_train, y_preds, output_dict=True)
pd.DataFrame(classrep)

,0,1,accuracy,macro avg,weighted avg
precision,0.953271,0.994350,0.967871,0.973811,0.969026
recall,0.996743,0.921466,0.967871,0.959104,0.967871
f1-score,0.974522,0.956522,0.967871,0.965522,0.967618
support,307.000000,191.000000,0.967871,498.000000,498.000000


In [25]:
cm = confusion_matrix(y_train, y_preds)
rubric = pd.DataFrame( {'pred_0': ['True Negative', 'False Negative'],
                        'pred_1': ['False Positive', 'True Positive']},
                      index=['actual_0', 'actual_1'])

rubric + ':' + cm.astype(str)

,pred_0,pred_1
actual_0,True Negative:306,False Positive:1
actual_1,False Negative:15,True Positive:176


#### Exercise 3
Print and clearly label the following: Accuracy, true positive rate, false positive rate, true negative rate, false negative rate, precision, recall, f1-score, and support.

In [26]:
 #returns an array from my confusion matrix
TN, FP, FN, TP = cm.ravel()

All = TP + TN + FP +FN
acc = (TP + TN) / All
recall = TP/ (TP +FN)
precision = TP / (TP + FP)
FPR = false_pos_rate = FP/ (FP + TN)
TNR = true_neg_rate = TN/ (TN +FP)
FNR = false_neg_rate = FN/ (FN +TP)
f1_score = 2 * ((precision*recall)/ (precision+recall))
support_pos = TP + FN
support_neg = FP + TN

In [27]:
print(f'''Accuracy: {acc:.2%},
Recall: {recall:.2%},
Precision: {precision:.2%},
False Positive Rate: {FPR:.2%},
True Negative Rate: {TNR:.2%},
False Negative Rate: {FNR:.2%},
F-1 Score: {f1_score:.2%},
Positive Support: {support_pos},
Negative Support: {support_neg}''')

Accuracy: 96.79%,
Recall: 92.15%,
Precision: 99.44%,
False Positive Rate: 0.33%,
True Negative Rate: 99.67%,
False Negative Rate: 7.85%,
F-1 Score: 95.65%,
Positive Support: 191,
Negative Support: 307


#### Exercise 4
Run through steps increasing your min_samples_leaf and decreasing your max_depth.


In [48]:
def rf_comp(X_train, y_train, X_val, y_val):
    little_john=[]
    # set range for max_Depth starting at 1, up to 15, counting by 2
    for i in range(1,15,2):
    # set range forin_samples starting at 3, up to 20, counting by 3
        for x in range(3,20,3):
    # fit a Random Forest classifier
            sherwood = RandomForestClassifier(max_depth= i, min_samples_leaf= x, random_state=123)

            rftestfit = sherwood.fit(X_train, y_train)

    # make predictions on the test set
            rftest_pred = sherwood.predict(X_train)

    # calculate model scores
            val_score = sherwood.score(X_val, y_val)
            train_score= sherwood.score(X_train, y_train)
            difference = round((train_score - val_score) * 100, 2)

            labels = {'max_depth': i,
                           'min_samples_leaf': x,
                           'Train Accuracy': train_score,
                           'Validate Accuracy': val_score,
                           'Percentage Difference': difference
                           }
    # create df that measures train score, test score, and the difference between them
            little_john.append(labels)
    return pd.DataFrame(little_john)

In [49]:
rf_comp(X_train, y_train, X_val, y_val)

,max_depth,min_samples_leaf,Train Accuracy,Validate Accuracy,Percentage Difference
0,1,3,0.799197,0.761682,3.75
1,1,6,0.799197,0.761682,3.75
2,1,9,0.799197,0.761682,3.75
3,1,12,0.799197,0.761682,3.75
4,1,15,0.803213,0.771028,3.22
5,1,18,0.803213,0.766355,3.69
6,3,3,0.815261,0.766355,4.89
7,3,6,0.817269,0.766355,5.09
8,3,9,0.817269,0.771028,4.62
9,3,12,0.817269,0.766355,5.09
